Market risk pySpark code

In [16]:
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean, lag, to_date
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.window import Window

# Initialize Spark session
spark = SparkSession.builder \
    .appName("MarketRiskEstimation") \
    .getOrCreate()

spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# Read the data
df = spark.read.csv('/content/googl_sp500_price.csv', header=True, inferSchema=True)

# Convert the date_gsheets column to date format
spark_df = df.withColumn('date_gsheets', to_date(col('date_gsheets'), 'MM/dd/yyyy HH:mm'))

# Calculate returns of GOOGL and the S&P500, use 'price_googl' and 'price_sp500'
window = Window.orderBy("date_gsheets")
returns_df = spark_df.withColumn("r_googl", (col("price_googl") - lag(col("price_googl")).over(window)) / lag(col("price_googl")).over(window)) \
    .withColumn("r_sp500", (col("price_sp500") - lag(col("price_sp500")).over(window)) / lag(col("price_sp500")).over(window))

# Get rid of NaN observations
returns_df = returns_df.dropna()

# Assemble features into a Vector
assembler = VectorAssembler(inputCols=["r_sp500"], outputCol="features")
data_with_features = assembler.transform(returns_df)

# Create a LinearRegression model and fit it to the data
lr = LinearRegression(featuresCol="features", labelCol="r_googl")
model = lr.fit(data_with_features)

# Extract the linear regression results
intercept = model.intercept
slope = model.coefficients[0]
summary = model.summary
pvalue = summary.pValues[0]
rvalue = summary.r2
stderr = summary.rootMeanSquaredError
# beta value
beta_googl = model.coefficients[0]

print(f"Linregress Result:")
print(f"Intercept: {intercept}")
print(f"P-value: {pvalue}")
print(f"R-value: {rvalue}")
print(f"Slope: {slope}")
print(f"Standard Error: {stderr}")
print(f"Beta: {beta_googl}")



Linregress Result:
Intercept: 0.0003255117119936872
P-value: 0.0
R-value: 0.31532484220905166
Slope: 1.0306293600577143
Standard Error: 0.011503042344037534
Beta: 1.0306293600577143
